In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.book import *
from nltk.corpus import stopwords
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import lightgbm as lgb
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm_notebook
import joblib
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from collections import defaultdict
from gensim import corpora
from gensim import models
import warnings
warnings.filterwarnings('ignore')

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [44]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
print(data)

           city  latitude  longitude                 attacktype1_txt  \
0      0.181904  0.681564   0.204755                   Armed Assault   
1      0.688351  0.688223   0.106546               Bombing/Explosion   
2      0.558561  0.732945   0.204053  Facility/Infrastructure Attack   
3      0.558561  0.732914   0.204130  Facility/Infrastructure Attack   
4      0.249211  0.704869   0.158102  Facility/Infrastructure Attack   
...         ...       ...        ...                             ...   
72545  0.009873  0.477141   0.603580               Bombing/Explosion   
72546  0.142187  0.611361   0.713903  Facility/Infrastructure Attack   
72547  0.787440  0.497927   0.597584               Bombing/Explosion   
72548  0.420521  0.660566   0.675203                   Armed Assault   
72549  0.974123  0.687351   0.180508                   Assassination   

                                        targsubtype1_txt  \
0        Police Building (headquarters, station, school)   
1              

In [45]:
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [46]:
# 无文本特征
import pygad

max_ = {'max': 0, 
           'learning_rate': 0,
           'min_child_samples': 0,              
           'max_depth': 0,
           'num_leaves': 0, 
           'colsample_bytree': 0,
           'reg_alpha': 0,
           'reg_lambda': 0}

def fitness_func(solution, solution_idx):
    # 从解决方案中提取超参数
    learning_rate = solution[0]
    min_child_samples = int(solution[1])
    max_depth = int(solution[2])
    num_leaves = int(solution[3])
    colsample_bytree = (solution[4])
    reg_alpha = solution[5]
    reg_lambda = solution[6]
    
    print(learning_rate, min_child_samples, max_depth, num_leaves,
          colsample_bytree, reg_alpha, reg_lambda)
    # 定义LightBGM的函数
    LGB = lgb.LGBMClassifier(learning_rate=learning_rate, # 学习率
                             min_child_samples=min_child_samples,
                             max_depth=max_depth, # 树的最大深度
                             num_leaves=num_leaves, 
                             colsample_bytree=colsample_bytree,
                             reg_alpha=reg_alpha,
                             reg_lambda=reg_lambda,
                             random_state=0 # 随机种子
                            )

    # 利用训练数据训练LightLGB分类器
    LGB.fit(X_train, y_train, categorical_feature=category_col)
    # 对测试数据进行预测
#     y_pred_prob = LGB.predict_proba(X_test)
    y_pred = LGB.predict(X_test)
    # 计算准确率
#     acc = accuracy_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
#     precision = precision_score(y_test, y_pred, average='weighted')
#     roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average="weighted")
#     print('当前准确率：', precision)
    # 返回适应函数分数（准确性）
    fitness = recall
    return fitness

# lightGbm
for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)

    # 确定超参数范围
    param_list = [
        {'low': 0.01, 'high': 1}, # learning_rate 
        {'low': 5, 'high': 80}, # min_child_samples
        {'low': 1, 'high': 50}, # max_depth 
        {'low': 5, 'high': 50}, #num_leaves 
        {'low': 0.1, 'high': 1}, #colsample_bytree
        {'low': 0, 'high': 100}, # reg_alpha
        {'low': 0, 'high': 100}, #reg_lambda
    ]


    # 定义遗传算法
    ga_instance = pygad.GA(num_generations=50, # 遗传算法的代数 
                           num_parents_mating=4, # 每代选择交叉的父代数量 
                           fitness_func=fitness_func, # 适应函数 
#                            initial_population=[20, 8],
                           sol_per_pop=15, # 种群中的解决方案数量 
                           num_genes=len(param_list), # 解决方案中的基因数量（即超参数数量） 
                           gene_type=float, # 基因类型（即超参数类型） 
                           gene_space=param_list, # 基因空间（即超参数范围） 
                           parent_selection_type='rws', # 父代选择类型 
                           keep_parents=1, # 保留的父代数量 
                           crossover_type='uniform', # 交叉类型 
                           crossover_probability=0.6,
                           mutation_type='random', # 变异类型 
                           mutation_probability=0.01,
#                            mutation_percent_genes=10 # 变异基因百分比
                           random_seed=0
                          )
    # 开始遗传算法
    ga_instance.run()

    # 获取最优超参数组合
    best_solution, best_fitness, best_solution_idx = ga_instance.best_solution()
    best_learning_rate = best_solution[0] 
    best_min_child_samples = int(best_solution[1])
    best_max_depth = int(best_solution[2])
    best_num_leaves = int(best_solution[3])
    best_colsample_bytree = (best_solution[4])
    best_reg_alpha = best_solution[5]
    best_reg_lambda = best_solution[6]

    # 打印最佳解决方案和最佳适应值
    print('Best solution is {solution} with fitness value {fitness}'.format(solution=best_solution, fitness=best_fitness)) 
    print('Best learning rate is {lr}'.format(lr=best_learning_rate)) 
    print('Best min child samples is {mcs}'.format(mcs=best_min_child_samples))
    print('Best max depth is {md}'.format(md=best_max_depth)) 
    print('Best num leaves is {nl}'.format(nl=best_num_leaves))  
    print('Best colsample bytree is {cb}'.format(cb=best_colsample_bytree))
    print('Best reg_alpha is {al}'.format(al=best_reg_alpha))
    print('Best reg_lambda is {la}'.format(la=best_reg_lambda))
    
    if best_fitness > max_['max']:
        max_['max'] = best_fitness
        max_['learning_rate'] = best_learning_rate 
        max_['min_child_samples'] = best_min_child_samples
        max_['max_depth'] = best_max_depth
        max_['num_leaves'] = best_num_leaves
        max_['colsample_bytree'] = best_colsample_bytree
        max_['reg_alpha'] = best_reg_alpha
        max_['reg_lambda'] = best_reg_lambda

print(max_)

第 1 折

0.5533253688880515 58 30 29 0.4812893194050143 64.58941130666561 43.75872112626925
0.892855270774259 77 19 40 0.5760054277776141 56.80445610939323 92.5596638292661
0.08032569761590806 11 1 42 0.8003410758548655 87.00121482468191 97.8618342232764
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.526629838532571 36 13 39 0.5105352989948937 56.84339488686485 1.8789800436355142
0.6214591421051183 50 31 47 0.7136382691931351 35.9507900573786 43.703195379934144
0.7006548839679922 9 33 35 0.2893443049664568 12.892629765485331 31.542835092418386
0.3700736632331964 47 22 49 0.19184032967322528 20.88767560948347 16.130951788499626
0.6565772422107444 23 23 15 0.24307262528096776 11.037514116430513 65.63295894652734
0.14680112183512767 19 19 41 0.18739114821375513 83.7944907498804 9.609840789396307
0.9766948703632617 40 48 32 0.7653372214584715 3.9187792254320675 28.280696257640958
0.1289945956010372 27 6 19 0.472836695063203 6.4147496348784365 69.24721193

0.892855270774259 77 19 35 0.6759289191947714 92.92961975762141 92.5596638292661
0.892855270774259 77 10 40 0.6759289191947714 92.92961975762141 92.5596638292661
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 77 19 40 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 77 20 10 0.6759289191947714 92.92961975762141 94.46689170495839
0.892855270774259 77 19 40 0.6759289191947714 92.92961975762141 94.46689170495839
0.892855270774259 39 19 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.892855270774259 39 19 40 0.6759289191947714 92.92961975762141 92.5596638292661
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 77 19 40 0.6759289191947714 14.33532874090464 92.5596638292661
0.8011669785745563 77 19 40 0.6759289191947714 14.33532874090464 92.5596638292661
0.8011669785745563 77 19 40 0.6759289191947714 14.33532874090464 92.5596638292661
0.8011669785745

0.8011669785745563 38 39 36 0.6759289191947714 14.33532874090464 62.64295646227256
0.8011669785745563 38 39 40 0.6759289191947714 14.33532874090464 92.5596638292661
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 38 39 40 0.3281919326907917 34.223582052824796 62.64295646227256
0.8011669785745563 38 39 40 0.6759289191947714 14.33532874090464 92.5596638292661
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 92.5596638292661
0.8011669785745563 38 39 40 0.3281919326907917 14.33532874090464 62.64295646227256
0.8011669785745563 67 39 40 0.3281919326907917 14.33532874090464 92.5596638292661
0.8011669785745563 39 39 10 0.3281919326907917 34.223582052824796 62.64295646227256
0.8011669785745563 39 39 40 0.3281919326907917 14.33532874090464 92.5596638292661
0.8011669785745563 39 39 40 0.6759289191947714 14.33532874090464 92.5596638292661
0.8011669785745563 38 39 40 0.3281919326907917 14.33532874090464 62.64295646227256
0.801166

0.6214591421051183 50 31 47 0.7136382691931351 35.9507900573786 43.703195379934144
0.5709354396645094 24 26 9 0.5760054277776141 92.92961975762141 31.856895245132367
0.892855270774259 77 19 9 0.5760054277776141 56.80445610939323 31.856895245132367
0.5709354396645094 24 26 9 0.5760054277776141 92.92961975762141 31.856895245132367
0.892855270774259 77 19 40 0.5760054277776141 92.92961975762141 92.5596638292661
0.5709354396645094 77 19 9 0.5760054277776141 92.92961975762141 31.856895245132367
0.892855270774259 77 26 9 0.6183518460005614 92.92961975762141 31.856895245132367
0.892855270774259 24 26 40 0.6183518460005614 56.80445610939323 92.5596638292661
0.5709354396645094 77 19 40 0.5760054277776141 56.80445610939323 92.5596638292661
0.5709354396645094 77 19 40 0.5760054277776141 92.92961975762141 92.5596638292661
0.5709354396645094 50 19 47 0.7136382691931351 92.92961975762141 43.703195379934144
0.5709354396645094 77 31 9 0.7136382691931351 35.9507900573786 31.856895245132367
0.8928552707

0.892855270774259 50 19 37 0.7136382691931351 92.92961975762141 43.703195379934144
0.892855270774259 37 31 37 0.7136382691931351 92.92961975762141 43.703195379934144
0.6214591421051183 50 31 47 0.7136382691931351 92.92961975762141 43.703195379934144
0.35876519841537735 50 19 47 0.7136382691931351 35.9507900573786 43.703195379934144
0.35876519841537735 50 31 37 0.7136382691931351 92.92961975762141 43.703195379934144
0.35876519841537735 50 31 9 0.7136382691931351 92.92961975762141 43.703195379934144
0.6214591421051183 50 19 37 0.7136382691931351 35.9507900573786 43.703195379934144
0.6214591421051183 50 31 37 0.7136382691931351 92.92961975762141 43.703195379934144
0.35876519841537735 50 31 47 0.7136382691931351 35.9507900573786 43.703195379934144
0.6214591421051183 50 31 47 0.7136382691931351 35.9507900573786 43.703195379934144
0.6214591421051183 50 19 37 0.7136382691931351 92.92961975762141 43.703195379934144
0.35876519841537735 50 19 37 0.7136382691931351 35.9507900573786 43.70319537993

...

0.526629838532571 36 13 39 0.5105352989948937 92.92961975762141 1.8789800436355142
0.526629838532571 39 13 10 0.6759289191947714 92.92961975762141 94.46689170495839
0.526629838532571 36 39 9 0.6759289191947714 14.33532874090464 1.8789800436355142
0.526629838532571 36 13 12 0.5105352989948937 92.92961975762141 1.8789800436355142
0.8011669785745563 39 13 10 0.5105352989948937 92.92961975762141 1.8789800436355142
0.526629838532571 39 13 10 0.5105352989948937 14.33532874090464 1.8789800436355142
0.526629838532571 36 39 10 0.5105352989948937 14.33532874090464 1.8789800436355142
0.8011669785745563 36 39 10 0.5105352989948937 92.92961975762141 94.46689170495839
0.526629838532571 36 39 9 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.526629838532571 36 13 10 0.5105352989948937 51.122178757368665 1.8789800436355142
0.526629838532571 36 13 9 0.5105352989948937 27.83252983201143 1.8789800436355142
0.52662

0.8011669785745563 39 39 35 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 35 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 12 0.6759289191947714 14.33532874090464 1.8789800436355142
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 37 0.6759289191947714 56.84339488686485 1.8789800436355142
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 37 0.6759289191947714 56.84339488686485 9.817215345644536
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.35876519841537735 39 39 37 0.6759289191947714 56.84339488686485 1.8789800436355142
0.8011669785745563 37 39 37 0.6759289191947714 56.84339488686485 1.8789800436355142
0.8011669785745563 39 39 10 0.6759289191947714 56.84339488686485 1.8789800436355142
0.8011669785745563 39 39 12 0.6759289191947714 14.33532874090464 94.4668917049583

0.8011669785745563 39 8 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.09651915251544743 39 8 12 0.3281919326907917 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 12 0.3281919326907917 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.3281919326907917 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 12 0.3281919326907917 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 12 0.6759289191947714 14.33532874090464 94.46689170495839
Best solution is [ 0.80116698 39.61095217 39.24592964 12.96526017  0.67592892 14.33532874
 94.4668917 ] with fitness value 0.6719503790489317
Best learning rate is 0.8011669785745563
Best min child samples is 39
Best max depth is 39
Best num leaves is 12
Best colsample bytree is 0.6759289191947714
Best reg_alpha is 14.33532874090464
Best reg_lambda is 94.46689170495839
第 9 折

0.5533253688880515 58 30 29 0.4812893194050143 64.58941130666561 43.75872112626925
0.892855270774259 77 19 40 0.576005427777

0.8011669785745563 76 35 12 0.6183518460005614 92.92961975762141 69.24721193700198
0.1289945956010372 39 39 12 0.6759289191947714 92.92961975762141 69.24721193700198
0.8011669785745563 39 39 12 0.6183518460005614 15.684301490151508 69.24721193700198
0.8011669785745563 39 39 12 0.6183518460005614 92.92961975762141 94.46689170495839
0.8011669785745563 76 39 12 0.6183518460005614 92.92961975762141 69.24721193700198
0.8011669785745563 39 39 10 0.6759289191947714 92.92961975762141 69.24721193700198
0.8011669785745563 39 39 16 0.6759289191947714 92.92961975762141 94.46689170495839
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 12 0.6759289191947714 92.92961975762141 94.46689170495839
0.1289945956010372 39 39 16 0.6759289191947714 92.92961975762141 94.46689170495839
0.8011669785745563 39 39 12 0.6759289191947714 92.92961975762141 69.24721193700198
0.1289945956010372 39 39 16 0.6759289191947714 92.92961975762141 94.46689170495839
0.7

0.8011669785745563 39 39 5 0.7006997866727364 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.5194139752374953 14.33532874090464 57.94142418856143
0.8011669785745563 39 39 10 0.24812666211143078 14.33532874090464 57.94142418856143
0.8011669785745563 39 39 10 0.7006997866727364 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.3365950191427438 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 24.057253304867253
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 57.94142418856143
0.8011669785745563 39 39 10 0.6759289191947714 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.3281919326907917 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.3281919326907917 14.33532874090464 94.46689170495839
0.8011669785745563 39 39 10 0.3281919326907917 14.33532874090464 87.5178149007563
0.8011669785745563 39 39 10 0.3281919326907917 14.33532874090464 26.83530495214004
0.80

In [47]:
{'max': 0.6736018596969351, 'learning_rate': 0.8011669785745563, 'min_child_samples': 25, 'max_depth': 23, 'num_leaves': 10, 'colsample_bytree': 0.6759289191947714, 'reg_alpha': 14.33532874090464, 'reg_lambda': 39.26967668741066}

{'max': 0.6736018596969351,
 'learning_rate': 0.8011669785745563,
 'min_child_samples': 25,
 'max_depth': 23,
 'num_leaves': 10,
 'colsample_bytree': 0.6759289191947714,
 'reg_alpha': 14.33532874090464,
 'reg_lambda': 39.26967668741066}

In [48]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [49]:
#lightGbm
import lightgbm as lgb

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []

max_roc_scores = []
max_acc_scores = []
max_f1_scores = []
max_recall_scores = []
max_precision_scores = []

lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    
    LGB = lgb.LGBMClassifier(random_state=0,
                             learning_rate=max_['learning_rate'],  
                             min_child_samples=max_['min_child_samples'],
                             max_depth=max_['max_depth'],
                             num_leaves=max_['num_leaves'],
                             colsample_bytree=max_['colsample_bytree'],
                             reg_alpha=max_['reg_alpha'], 
                             reg_lambda=max_['reg_lambda']
                            )
    LGB.fit(X_train, y_train, categorical_feature=category_col)
    lgb_feature_importances[f'fold_{k+1}'] = LGB.feature_importances_
    y_pred_prob = LGB.predict_proba(X_test)
    y_pred = LGB.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    lgb_f1_scores.append(f1)
    lgb_roc_scores.append(roc_auc)
    lgb_acc_scores.append(acc)
    lgb_recall_scores.append(recall)
    lgb_precision_scores.append(precision)
    max_roc_scores = max(lgb_roc_scores)
    max_acc_scores = max(lgb_acc_scores)
    max_f1_scores = max(lgb_f1_scores)
    max_recall_scores = max(lgb_recall_scores)
    max_precision_scores = max(lgb_precision_scores)
    
print(f'average roc score: {max_roc_scores}')
print(f'average acc_score: {max_acc_scores}')
print(f'average f1_score: {max_f1_scores}')
print(f'average recall_score: {max_recall_scores}')
print(f'average precision_score: {max_precision_scores}')

第 1 折

 Fold 1 | 
 AUC_ROC: 86.25217578946418%
 ACC: 66.02343211578223%
 F1: 65.57967676918356%
 RECALL: 66.02343211578223%
 PRECISION: 65.50368919802439%
第 2 折

 Fold 2 | 
 AUC_ROC: 86.38261895014153%
 ACC: 65.6788421778084%
 F1: 65.21456053289336%
 RECALL: 65.6788421778084%
 PRECISION: 65.17437322075308%
第 3 折

 Fold 3 | 
 AUC_ROC: 86.70820708270287%
 ACC: 66.83666436940041%
 F1: 66.38857255582865%
 RECALL: 66.83666436940041%
 PRECISION: 66.39075319659568%
第 4 折

 Fold 4 | 
 AUC_ROC: 86.91788353814485%
 ACC: 66.79531357684355%
 F1: 66.27353827574561%
 RECALL: 66.79531357684355%
 PRECISION: 66.15403181640508%
第 5 折

 Fold 5 | 
 AUC_ROC: 86.40180278848362%
 ACC: 66.65747760165404%
 F1: 66.19175583439343%
 RECALL: 66.65747760165404%
 PRECISION: 66.18516721480201%
第 6 折

 Fold 6 | 
 AUC_ROC: 86.61112351581315%
 ACC: 66.25775327360441%
 F1: 65.92799273030265%
 RECALL: 66.25775327360441%
 PRECISION: 65.961446854454%
第 7 折

 Fold 7 | 
 AUC_ROC: 86.41256481383255%
 ACC: 66.35423845623708%
 F